In [ ]:
#default_exp build_lib

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *
import pprint

## Parse Open API Spec

This library leverages the [OpenAPI Specification](https://github.com/OAI/OpenAPI-Specification) to create a python client for the GitHub API.  The OpenAPI specification contains metadata on all of the endpoints and how to access them properly.  Using this metadata, we can construct a python client dynamically that updates automatically along with the OpenAPI Spec. 

In [ ]:
#export
GH_OPENAPI_URL = 'https://github.com/github/rest-api-description/raw/main/descriptions/api.github.com/api.github.com.json?raw=true'
_DOC_URL = 'https://docs.github.com/'

In [ ]:
#export
def build_funcs(pre, nm='ghapi/metadata.py', url=GH_OPENAPI_URL, docurl=_DOC_URL):
    "Build module funcs.py from an Open API spec and optionally filter by a path `pre`"
    def _get_detls(o):
        data = nested_idx(o, *'requestBody content application/json schema properties'.split()) or {}
        url = o['externalDocs']['url'][len(docurl):]
        return (o['operationId'], o['summary'], url, list(data.keys()))
    
    js = urljson(url)
    paths = {o[len(pre):]:v for o,v in js['paths'].items() if o.startswith(pre)}
    _funcs = [(path, verb) + _get_detls(detls)
              for path,verbs in paths.items() for verb,detls in verbs.items()]
    Path(nm).write_text("funcs = " + pprint.pformat(_funcs, width=360))

We build the module `metadata.py` with the github OpenAPI spec for only paths beginning with `/repos/{owner}/{repo}`:

```python
build_funcs(pre='/repos/{owner}/{repo}', nm='ghapi/metadata.py')
```

This python module contains a list of metadata for each endpoint, containing the path, verb, operation id, summary, documentation relative URL, and list of parameters (if any), e.g:

In [ ]:
from ghapi.metadata import funcs
funcs[5]

('/actions/artifacts/{artifact_id}',
 'delete',
 'actions/delete-artifact',
 'Delete an artifact',
 'rest/reference/actions#delete-an-artifact',
 [])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script('build_lib.ipynb')